# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Bouwman  ->  J. Bouwman  |  ['J. Bouwman']
C. Danti  ->  C. Danti  |  ['C. Danti']
J. Mah  ->  J. Mah  |  ['J. Mah']
Arxiv has 70 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2310.02795


extracting tarball to tmp_2310.02795...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3526: LatexWarning: Found documentclass in tmp_2310.02795/etacha_29_09_2023.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

Found 133 bibliographic references in tmp_2310.02795/etacha_29_09_2023.bbl.
Retrieving document from  https://arxiv.org/e-print/2310.02886


extracting tarball to tmp_2310.02886...

 done.


C. Danti  ->  C. Danti  |  ['C. Danti']
J. Mah  ->  J. Mah  |  ['J. Mah']


list index out of range


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.02795-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.02795) | **Protoplanetary and debris disks in the $η$ Chamaeleontis Association:  A sub-millimeter survey obtained with APEX/LABOCA**  |
|| V. Roccatagliata, et al. -- incl., <mark>J. Bouwman</mark> |
|*Appeared on*| *2023-10-05*|
|*Comments*| *20 pages, 9 figures, 7 tables*|
|**Abstract**| Nearby associations are ideal regions to study coeval samples of protoplanetary and debris disks down to late M-type stars. Those aged 5-10,Myrs, where most of the disk should have already dissipated forming planets, are of particular interest. We present the first complete study of both protoplanetary and debris disks in a young region, using the $\eta$ Chamaeleontis association as a test bench to study the cold disk content. We obtained sub-millimeter data for the entire core population down to late M-type stars, plus a few halo members. We performed a continuum sub-millimeter survey with APEX/LABOCA of all the core populations of $\eta$ Cha association. Disk properties have been derived by modeling protoplanetary and debris disks using RADMC 2D and DMS, respectively. We find that protoplanetary disks in $\eta$ Cha typically have holes with radii of the order of 0.01 to 0.03 AU, while ring-like emission from the debris disks is located between 20 au and 650 au from the central star. The parallaxes and Gaia eDR3 photometry, in combination with the PARSEC and COLIBRI isochrones, enable us to confirm an age of $\eta$ Cha between 7 and 9 Myrs. In general, the disk mass seems insufficient to support accretion over a long time, even for the lowest mass accretors, a clear difference compared with other regions and also a sign that the mass budget is further underestimated. We do not find a correlation between the stellar masses, accretion rates, and disk masses, although this could be due to sample issues. We confirm that the presence of inner holes is not enough to stop accretion unless accompanied by dramatic changes to the total disk mass content. Comparing $\eta$ Cha with other regions at different ages, we find that the physical processes responsible for debris disks (e.g., dust growth, dust trapping) efficiently act in less than 5 Myrs. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2310.02886-b31b1b.svg)](https://arxiv.org/abs/arXiv:2310.02886) | **Composition of giant planets: the roles of pebbles and planetesimals**  |
|| <mark>C. Danti</mark>, B. Bitsch, <mark>J. Mah</mark> |
|*Appeared on*| *2023-10-05*|
|*Comments*| *12 pages, 10 figures, accepted for publication in Astronomy and Astrophysics*|
|**Abstract**| One of the current challenges of planet formation theory is to explain the enrichment of observed exoplanetary atmospheres. Past studies have focused on scenarios where either pebbles or planetesimals were the heavy element enrichment's drivers, we combine here both approaches to understand whether the composition of a planet can constrain its formation pathway. We study three different formation scenarios: pebble accretion, pebble accretion with planetesimal formation, combined pebble and planetesimal accretion. We use the chemcomp code to perform semi-analytical 1D simulations of protoplanetary discs, including viscous evolution, pebble drift, and simple chemistry to simulate the growth of planets from planetary embryos to gas giants as they migrate through the disc, while tracking their composition. Our simulations confirm that the composition of the planetary atmosphere is dominated by the accretion of gas enriched by inward drifting and evaporating pebbles. Including planetesimal formation hinders the enrichment, because the pebbles locked into planetesimals cannot evaporate and enrich the disc. This results in a big drop of the accreted heavy elements both in the planetesimal formation and accretion case, proving that planetesimal formation needs to be inefficient in order to explain planets with high heavy element content. Accretion of planetesimals enhances the refractory component of the atmosphere, leading to low volatile to refractory ratios, contrary to the pure pebble scenario. Such low volatile to refractory ratios can also be achieved by planets migrating in the inner disc in pure pebble scenario. Distinguishing these two scenarios requires knowledge about the planet's atmospheric C/H and O/H ratios, which are higher for pure pebble accretion. Therefore, a detailed knowledge of the composition of planetary atmospheres could help to constrain the planet's formation pathway. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2310.02795.md
    + _build/html/tmp_2310.02795/./Sptype_Mdot_2023.png
    + _build/html/tmp_2310.02795/./Mdot_Mdisk_2023.png
    + _build/html/tmp_2310.02795/./recx5_sed_models.png
    + _build/html/tmp_2310.02795/./recx9_sed_models.png
    + _build/html/tmp_2310.02795/./recx11_sed_models.png
    + _build/html/tmp_2310.02795/./echa0801_sed_models.png
    + _build/html/tmp_2310.02795/./echa0820_sed_models.png
    + _build/html/tmp_2310.02795/./echa0841_sed_models.png
    + _build/html/tmp_2310.02795/./echa0843_sed_models.png
    + _build/html/tmp_2310.02795/./echa0844_sed_models.png
    + _build/html/tmp_2310.02795/./Sptype_apex_2023b.png
exported in  _build/html/2310.02886.md
    + _build/html/tmp_2310.02886/./atm_comp_growth_track.png
    + _build/html/tmp_2310.02886/./mass.png
    + _build/html/tmp_2310.02886/./surface_densities_viscosities.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Protoplanetary and debris disks in the $\eta$ Chamaeleontis Association:

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.02795-b31b1b.svg)](https://arxiv.org/abs/2310.02795)<mark>Appeared on: 2023-10-05</mark> -  _20 pages, 9 figures, 7 tables_

</div>
<div id="authors">

V. Roccatagliata, et al. -- incl., <mark>J. Bouwman</mark>

</div>
<div id="abstract">

**Abstract:** Nearby associations are ideal regions to study coeval samples of protoplanetary and debris disks down to late M-type stars. Those aged 5-10,Myrs, where most of the disk should have already dissipated forming planets, are of particular interest. We present the first complete study of both protoplanetary and debris disks in a young region, using the $\eta$ Chamaeleontis ( $\eta$ Cha) association as a test bench to study the cold disk content. We obtained sub-millimeter data for the entire core population down to late M-type stars, plus a few halo members. We performed a continuum sub-millimeter survey with APEX/LABOCA of all the core populations of $\eta$ Cha association. These data are combined with archival, multi-wavelength photometry to compile a complete spectral energy distribution. Disk properties have been derived by modeling protoplanetary and debris disks using RADMC 2D and DMS, respectively. We compute a lower limit of the disk millimeter fraction, which is then compared to the corresponding disk fraction in the infrared for $\eta$ Cha. We also revisit and refine the age estimate for the region, using the Gaia eDR3 astrometry and photometry for the core sources. We find that protoplanetary disks in $\eta$ Cha typically have holes with radii of the order of 0.01 to 0.03 AU, while ring-like emission from the debris disks is located between 20 au and 650 au from the central star. The parallaxes and Gaia eDR3 photometry, in combination with the PARSEC and COLIBRI  isochrones, enable us to confirm an age of $\eta$ Cha between 7 and 9 Myrs. In general, the disk mass seems insufficient to support accretion over a long time, even for the lowest mass accretors, a clear difference compared with other regions and also a sign that the mass budget is further underestimated. We do not find a correlation between the stellar masses, accretion rates, and disk masses, although this could be due to sample issues (the objects are few and mostly low-mass). We confirm that the presence of inner holes is not enough to stop accretion unless accompanied by dramatic changes to the total disk mass content. Comparing $\eta$ Cha with other regions at different ages, we find that the physical processes responsible for debris disks (e.g., dust growth, dust trapping) efficiently act in less than 5 Myrs.

</div>

<div id="div_fig1">

<img src="tmp_2310.02795/./Sptype_Mdot_2023.png" alt="Fig4.1" width="50%"/><img src="tmp_2310.02795/./Mdot_Mdisk_2023.png" alt="Fig4.2" width="50%"/>

**Figure 4. -** _ Top_: Accretion rate vs spectral type. Note the lack of significant correlations between these quantities. _ Bottom_: Circumstellar dust mass derived according to Eqn. \ref{mdust-eq} vs. accretion rate.  For comparison, the objects in the list from [Manara, Ansdell and Rosotti (2023)]() are shown in grey. Note that the total disk mass would depend on the gas-to-dust ratio. APEX flux and mass upper limits are marked by inverted triangles. The red line corresponds to the model fitted by [Manara, Rosotti and Testi (2016)](). The accretion rates are taken from the literature (see Table \ref{macc}). For each object, we show the individual measurements (small symbols) together with the average rate (large symbols). A further color ring is added to specify the type of disk according to the classical SED classification.  (*mdotmdisk-fig*)

</div>
<div id="div_fig2">

<img src="tmp_2310.02795/./recx5_sed_models.png" alt="Fig7.1" width="12%"/><img src="tmp_2310.02795/./recx9_sed_models.png" alt="Fig7.2" width="12%"/><img src="tmp_2310.02795/./recx11_sed_models.png" alt="Fig7.3" width="12%"/><img src="tmp_2310.02795/./echa0801_sed_models.png" alt="Fig7.4" width="12%"/><img src="tmp_2310.02795/./echa0820_sed_models.png" alt="Fig7.5" width="12%"/><img src="tmp_2310.02795/./echa0841_sed_models.png" alt="Fig7.6" width="12%"/><img src="tmp_2310.02795/./echa0843_sed_models.png" alt="Fig7.7" width="12%"/><img src="tmp_2310.02795/./echa0844_sed_models.png" alt="Fig7.8" width="12%"/>

**Figure 7. -** SED models for the protoplanetary disks. For all cases, the photometry data is marked by blue circles (empty symbols for marginal detections). Spitzer/IRS spectra are plotted in black. The MARCS model stellar photospheres are marked by dotted grey lines, and the various disk models are represented by colored lines and labeled according to their main characteristics (see Table \ref{models-table}). RECX-5: A large-scale gap is not needed to reproduce the long wavelengths, which also means that SED alone cannot constrain among many diverse but equally plausible SED structures. RECX-9: A change in vertical scale height at 10-15 au is needed, which could be caused by a gap, wall, warp, or any other structure affecting the density and the scale height probably created by the existing companion at 20 au. RECX-11: Best fit with relatively massive and flattened disks. A more puffed innermost disk (either a puffed-up rim or a more extended $\sim$0.6 au region) is required, with the disk becoming increasingly flattened and settled at larger radii. J0801 and J0820 appear to be examples of relatively massive transition disks with large, strongly mass-depleted inner holes. ECHA J0841: Very flattened SED. ECHA J0843: Small gap or hole required. For ECHA J0844: Gap and/or change in the vertical scale height needed to explain the far-IR flux.  (*models-fig*)

</div>
<div id="div_fig3">

<img src="tmp_2310.02795/./Sptype_apex_2023b.png" alt="Fig1" width="100%"/>

**Figure 1. -** APEX flux versus spectral type for the $\eta$ Cha members. For comparison, the objects in the list from [Manara, Ansdell and Rosotti (2023)]() are shown in grey, scaled to the distance of $\eta$ Cha. Note the lack of significant correlations between these quantities. Filled dots represent flux detections, while inverted triangles show upper limits. A further color ring is added to specify the type of disk according to the classical SED classification adopted by [Sicilia-Aguilar, Bouwman and Juhász (2009)](): transitional disks (TD), Class II, and Class III (including the debris disks). RS Cha and $\eta$ Cha are considered as upper limits due to cloud contamination.
 (*sptmdisk-fig*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.02795"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# Composition of giant planets: the roles of pebbles and planetesimals

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2310.02886-b31b1b.svg)](https://arxiv.org/abs/2310.02886)<mark>Appeared on: 2023-10-05</mark> -  _12 pages, 10 figures, accepted for publication in Astronomy and Astrophysics_

</div>
<div id="authors">

<mark>C. Danti</mark>, B. Bitsch, <mark>J. Mah</mark>

</div>
<div id="abstract">

**Abstract:** One of the current challenges of planet formation theory is to explain the enrichment of observed exoplanetary atmospheres.   While past studies have focused on scenarios where either pebbles or planetesimals are the main drivers of heavy element enrichment, we combine here both approaches to understand whether the composition of a planet can constrain its formation pathway. We study three different formation scenarios: pebble accretion, pebble accretion with planetesimal formation inside the disc, combined pebble and planetesimal accretion.   We use the chemcomp code to perform semi-analytical 1D simulations of protoplanetary discs, including viscous evolution, pebble drift, and simple chemistry to simulate the growth of planets from planetary embryos to gas giants as they migrate through the disc, while simultaneously tracking their composition.   Our simulations confirm that the composition of the planetary atmosphere is dominated by the accretion of gas vapour enriched by inward drifting and evaporating pebbles. Including planetesimal formation hinders this enrichment, because many pebbles are locked into planetesimals and cannot evaporate and enrich the disc. This results in a dramatic drop of the accreted heavy elements both in the planetesimal formation and accretion case, demonstrating that planetesimal formation needs to be inefficient in order to explain planets with high heavy element content.   On the other hand, accretion of planetesimals enhances the refractory component of the atmosphere, leading to low volatile to refractory ratios in the atmosphere, in contrast to the majority of pure pebble simulations. However, low volatile to refractory ratios can also be achieved in the pure pebble accretion scenario, if the planet migrates all the way into the inner disc and accretes gas that is enriched in evaporated refractories.   Distinguishing these two scenarios requires knowledge about the planet's atmospheric C/H and O/H ratios, which are much higher in the pure pebble scenario compared to the planetesimal formation and accretion scenario.   This implies that a detailed knowledge of the composition of planetary atmospheres could help to distinguish between the different formation scenarios.

</div>

<div id="div_fig1">

<img src="tmp_2310.02886/./atm_comp_growth_track.png" alt="Fig4" width="100%"/>

**Figure 4. -** Final elemental abundances of the planetary atmospheres (top) and their corresponding growth tracks (bottom) for the three different scenarios of only pebble accretion (left), planetesimal formation (middle) and pebble and planetesimal accretion (right). The horizontal blue line in the first row marks the solar abundance, while the vertical violet lines in the second row show the evaporation fronts of the chemical species included in our model for a disc viscosity of $\alpha = 5 \cdot 10^{-4}$.The solid lines of the growth tracks correspond to core formation, while the dotted lines correspond to the gas accretion phase. The different colour codings represent different disc viscosities. (*fig:atm_comp*)

</div>
<div id="div_fig2">

<img src="tmp_2310.02886/./mass.png" alt="Fig3" width="100%"/>

**Figure 3. -** Total mass of pebbles and planetesimal and water fraction in the gas in the disc. Left panel: Total mass of pebbles (light blue lines) and planetesimals (dark blue line) in the two scenarios: pebble accretion only (dotted line) and planetesimal formation (solid lines). Central and right panel: Water content in the gaseous phase of the disc with viscosity $\alpha=10^{-3}$ as a function of radius and time in the case of no planetesimal formation (central panel) and in presence of planetesimal formation (right panel). The vertical violet line marks the water evaporation front in the disc. (*fig:water_disk*)

</div>
<div id="div_fig3">

<img src="tmp_2310.02886/./surface_densities_viscosities.png" alt="Fig6" width="100%"/>

**Figure 6. -** Surface densities of gas, pebbles and planetesimals for the disc described in Table \ref{tab:param_disc} in absence of planets, for different disc viscosities increasing from left to right. The top panel shows the pebble accretion scenario, where planetesimals cannot form, the bottom panel shows what happens, instead, when planetesimal formation is involved. (*fig:surface_density*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2310.02886"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

288  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
